In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)

## Loading Dataset

In [9]:
data = pd.read_csv('H-1B_Disclosure_Data_FY2019.csv',low_memory=False)

In [10]:
data.head()

,CASE_NUMBER,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,ORIGINAL_CERT_DATE,VISA_CLASS,JOB_TITLE,SOC_CODE,SOC_TITLE,FULL_TIME_POSITION,PERIOD_OF_EMPLOYMENT_START_DATE,PERIOD_OF_EMPLOYMENT_END_DATE,TOTAL_WORKER_POSITIONS,NEW_EMPLOYMENT,CONTINUED_EMPLOYMENT,CHANGE_PREVIOUS_EMPLOYMENT,NEW_CONCURRENT_EMPLOYMENT,CHANGE_EMPLOYER,AMENDED_PETITION,EMPLOYER_NAME,EMPLOYER_BUSINESS_DBA,EMPLOYER_ADDRESS1,EMPLOYER_ADDRESS2,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_POSTAL_CODE,EMPLOYER_COUNTRY,EMPLOYER_PROVINCE,EMPLOYER_PHONE,EMPLOYER_PHONE_EXT,NAICS_CODE,AGENT_REPRESENTING_EMPLOYER,AGENT_ATTORNEY_LAW_FIRM_BUSINESS_NAME,AGENT_ATTORNEY_ADDRESS1,AGENT_ATTORNEY_ADDRESS2,AGENT_ATTORNEY_CITY,AGENT_ATTORNEY_STATE,AGENT_ATTORNEY_POSTAL_CODE,AGENT_ATTORNEY_COUNTRY,AGENT_ATTORNEY_PROVINCE,AGENT_ATTORNEY_PHONE,AGENT_ATTORNEY_PHONE_EXT,STATE_OF_HIGHEST_COURT,NAME_OF_HIGHEST_STATE_COURT,WORKSITE_WORKERS_1,SECONDARY_ENTITY_1,SECONDARY_ENTITY_BUSINESS_NAME_1,WORKSITE_ADDRESS1_1,WORKSITE_ADDRESS2_1,WORKSITE_CITY_1,WORKSITE_COUNTY_1,WORKSITE_STATE_1,WORKSITE_POSTAL_CODE_1,WAGE_RATE_OF_PAY_FROM_1,WAGE_RATE_OF_PAY_TO_1,WAGE_UNIT_OF_PAY_1,PREVAILING_WAGE_1,PW_UNIT_OF_PAY_1,PW_TRACKING_NUMBER_1,PW_WAGE_LEVEL_1,PW_OES_YEAR_1,PW_OTHER_SOURCE_1,PW_NON-OES_YEAR_1,PW_SURVEY_PUBLISHER_1,PW_SURVEY_NAME_1,WORKSITE_WORKERS_2,SECONDARY_ENTITY_2,SECONDARY_ENTITY_BUSINESS_NAME_2,WORKSITE_ADDRESS1_2,WORKSITE_ADDRESS2_2,WORKSITE_CITY_2,WORKSITE_COUNTY_2,WORKSITE_STATE_2,WORKSITE_POSTAL_CODE_2,WAGE_RATE_OF_PAY_FROM_2,WAGE_RATE_OF_PAY_TO_2,WAGE_UNIT_OF_PAY_2,PREVAILING_WAGE_2,PW_UNIT_OF_PAY_2,PW_TRACKING_NUMBER_2,PW_WAGE_LEVEL_2,PW_OES_YEAR_2,PW_OTHER_SOURCE_2,PW_NON-OES_YEAR_2,PW_SURVEY_PUBLISHER_2,PW_SURVEY_NAME_2,WORKSITE_WORKERS_3,SECONDARY_ENTITY_3,SECONDARY_ENTITY_BUSINESS_NAME_3,WORKSITE_ADDRESS1_3,WORKSITE_ADDRESS2_3,WORKSITE_CITY_3,WORKSITE_COUNTY_3,WORKSITE_STATE_3,WORKSITE_POSTAL_CODE_3,WAGE_RATE_OF_PAY_FROM_3,WAGE_RATE_OF_PAY_TO_3,WAGE_UNIT_OF_PAY_3,PREVAILING_WAGE_3,PW_UNIT_OF_PAY_3,PW_TRACKING_NUMBER_3,PW_WAGE_LEVEL_3,PW_OES_YEAR_3,PW_OTHER_SOURCE_3,PW_NON-OES_YEAR_3,PW_SURVEY_PUBLISHER_3,PW_SURVEY_NAME_3,WORKSITE_WORKERS_4,SECONDARY_ENTITY_4,SECONDARY_ENTITY_BUSINESS_NAME_4,WORKSITE_ADDRESS1_4,WORKSITE_ADDRESS2_4,WORKSITE_CITY_4,WORKSITE_COUNTY_4,WORKSITE_STATE_4,WORKSITE_POSTAL_CODE_4,WAGE_RATE_OF_PAY_FROM_4,WAGE_RATE_OF_PAY_TO_4,WAGE_UNIT_OF_PAY_4,PREVAILING_WAGE_4,PW_UNIT_OF_PAY_4,PW_TRACKING_NUMBER_4,PW_WAGE_LEVEL_4,PW_OES_YEAR_4,PW_OTHER_SOURCE_4,PW_NON-OES_YEAR_4,PW_SURVEY_PUBLISHER_4,PW_SURVEY_NAME_4,WORKSITE_WORKERS_5,SECONDARY_ENTITY_5,SECONDARY_ENTITY_BUSINESS_NAME_5,WORKSITE_ADDRESS1_5,WORKSITE_ADDRESS2_5,WORKSITE_CITY_5,WORKSITE_COUNTY_5,WORKSITE_STATE_5,WORKSITE_POSTAL_CODE_5,WAGE_RATE_OF_PAY_FROM_5,WAGE_RATE_OF_PAY_TO_5,WAGE_UNIT_OF_PAY_5,PREVAILING_WAGE_5,PW_UNIT_OF_PAY_5,PW_TRACKING_NUMBER_5,PW_WAGE_LEVEL_5,PW_OES_YEAR_5,PW_OTHER_SOURCE_5,PW_NON-OES_YEAR_5,PW_SURVEY_PUBLISHER_5,PW_SURVEY_NAME_5,WORKSITE_WORKERS_6,SECONDARY_ENTITY_6,SECONDARY_ENTITY_BUSINESS_NAME_6,WORKSITE_ADDRESS1_6,WORKSITE_ADDRESS2_6,WORKSITE_CITY_6,WORKSITE_COUNTY_6,WORKSITE_STATE_6,WORKSITE_POSTAL_CODE_6,WAGE_RATE_OF_PAY_FROM_6,WAGE_RATE_OF_PAY_TO_6,WAGE_UNIT_OF_PAY_6,PREVAILING_WAGE_6,PW_UNIT_OF_PAY_6,PW_TRACKING_NUMBER_6,PW_WAGE_LEVEL_6,PW_OES_YEAR_6,PW_OTHER_SOURCE_6,PW_NON-OES_YEAR_6,PW_SURVEY_PUBLISHER_6,PW_SURVEY_NAME_6,WORKSITE_WORKERS_7,SECONDARY_ENTITY_7,SECONDARY_ENTITY_BUSINESS_NAME_7,WORKSITE_ADDRESS1_7,WORKSITE_ADDRESS2_7,WORKSITE_CITY_7,WORKSITE_COUNTY_7,WORKSITE_STATE_7,WORKSITE_POSTAL_CODE_7,WAGE_RATE_OF_PAY_FROM_7,WAGE_RATE_OF_PAY_TO_7,WAGE_UNIT_OF_PAY_7,PREVAILING_WAGE_7,PW_UNIT_OF_PAY_7,PW_TRACKING_NUMBER_7,PW_WAGE_LEVEL_7,PW_OES_YEAR_7,PW_OTHER_SOURCE_7,PW_NON-OES_YEAR_7,PW_SURVEY_PUBLISHER_7,PW_SURVEY_NAME_7,WORKSITE_WORKERS_8,SECONDARY_ENTITY_8,SECONDARY_ENTITY_BUSINESS_NAME_8,WORKSITE_ADDRESS1_8,WORKSITE_ADDRESS2_8,WORKSITE_CITY_8,WORKSITE_COUNTY_8,WORKSITE_STATE_8,WORKSITE_POSTAL_CODE_8,WAGE_RATE_OF_PAY_FROM_8,WAGE_RATE_OF_PAY_TO_8,WAGE_UNIT_OF_PAY_8,PREVAI

In [11]:
data.columns

Index(['CASE_NUMBER', 'CASE_STATUS', 'CASE_SUBMITTED', 'DECISION_DATE',
       'ORIGINAL_CERT_DATE', 'VISA_CLASS', 'JOB_TITLE', 'SOC_CODE',
       'SOC_TITLE', 'FULL_TIME_POSITION',
       ...
       'PW_OTHER_SOURCE_10', 'PW_NON-OES_YEAR_10', 'PW_SURVEY_PUBLISHER_10',
       'PW_SURVEY_NAME_10', 'H-1B_DEPENDENT', 'WILLFUL_VIOLATOR',
       'SUPPORT_H1B', 'STATUTORY_BASIS', 'MASTERS_EXEMPTION',
       'PUBLIC_DISCLOSURE'],
      dtype='object', length=260)

In [13]:
percentage_null = (data.isna().mean() * 100)
null_value_column = []
print("Columns that are Missing values greater than 98% :")
for x,y in zip(percentage_null.index.tolist() , percentage_null.values):
    if y > 98:
        null_value_column.append(x)
print(null_value_column)

In [15]:
# Further Preprocessing
data.drop('CASE_NUMBER',axis=1,inplace=True)

#### Date and time conversion

In [16]:
data.CASE_SUBMITTED = pd.to_datetime(data.CASE_SUBMITTED)
data.DECISION_DATE = pd.to_datetime(data.DECISION_DATE)
data.ORIGINAL_CERT_DATE = pd.to_datetime(data.ORIGINAL_CERT_DATE)
data.PERIOD_OF_EMPLOYMENT_START_DATE = pd.to_datetime(data.PERIOD_OF_EMPLOYMENT_START_DATE)
data.PERIOD_OF_EMPLOYMENT_END_DATE = pd.to_datetime(data.PERIOD_OF_EMPLOYMENT_END_DATE)

In [17]:
data.groupby(by='CASE_STATUS').size()

CASE_STATUS
CERT                       1
CERTIFIED              42813
CERTIFIED-WITHDRAWN     3266
DENIED                   870
WITHDRAWN               1622
dtype: int64

We find there are `173566 Certified`, `1817 Denied`, `17926 Certified-withdrawn` and `11288 withdrawn` cases are on the datasets. The Certified-withdrawn, Withdrawn cases that are in this datasets are of no use as they do not play any role in Our model.

Now, Excluding the `Withdrawn` and `Certified-Withdrawn`,

In [1]:
# df['CASE_STATUS'] = df['CASE_STATUS'].replace({'CERTIFIED-WITHDRAWN':'CERTIFIED', 'WITHDRAWN':'DENIED'})

In [18]:
data.drop(data[data['CASE_STATUS']=='WITHDRAWN'].index,inplace=True)
data.drop(data[data['CASE_STATUS']=='CERTIFIED-WITHDRAWN'].index,inplace=True)

In [19]:
data.reset_index(inplace=True,drop=True)
data.shape

(43684, 259)

In [ ]:
data.dropna(subset=['CASE_STATUS'],inplace=True)

In [ ]:
selected_columns = [
    'CASE_STATUS','VISA_CLASS','FULL_TIME_POSITION','SECONDARY_ENTITY_1', 'AGENT_REPRESENTING_EMPLOYER', 'TOTAL_WORKER_POSITIONS',
    'NEW_EMPLOYMENT', 'CONTINUED_EMPLOYMENT', 'CHANGE_PREVIOUS_EMPLOYMENT', 'NEW_CONCURRENT_EMPLOYMENT',
    'CHANGE_EMPLOYER', 'AMENDED_PETITION','H-1B_DEPENDENT','WILLFUL_VIOLATOR','WAGE_RATE_OF_PAY_FROM_1',
    'WAGE_RATE_OF_PAY_TO_1','WAGE_UNIT_OF_PAY_1','PREVAILING_WAGE_1'
    ]
data_final = data[selected_columns].copy()

In [ ]:
def quantile_outlier_check(dataframe,columns=None):
    '''
    Returns the index of outlier columns.
    '''
    dataframe = dataframe.select_dtypes(exclude=['object','datetime'])
    if columns == None:
        columns = dataframe.columns.to_list()
    Q1 = dataframe.quantile(0.25)
    Q3 = dataframe.quantile(0.75)
    IQR = Q3 - Q1
    lower_limit = Q1 - 1.5*IQR
    upper_limit = Q3 + 1.5*IQR
    outlier_INDEX = set()
    for column in columns:
        outlier_INDEX.update(dataframe[(dataframe[column]<lower_limit[column]) | (dataframe[column]>upper_limit[column])].index.to_list())
    return outlier_INDEX

outlier = quantile_outlier_check(data_final,['TOTAL_WORKER_POSITIONS','CHANGE_PREVIOUS_EMPLOYMENT'])
len(outlier)

In [ ]:
data_final.drop(outlier,inplace=True)

def Status(row):
    if row['CASE_STATUS'] == 'CERTIFIED':
        return 1
    else:
        return 0
    
data_final['CASE_STATUS'] = data_final.apply(lambda rw: Status(rw),axis=1)

In [ ]:
tmp_data = data_final.select_dtypes(exclude=['datetime','object']).copy()
def corr_columns(data,threshold):
    temp = set()
    coef_corr = data.corr()
    for i in range(len(data.columns)):
        for j in range(i):
            if abs(coef_corr.iloc[i,j]) > threshold:
                temp.add(coef_corr.columns[i])
    return temp

# Checking the correlated columns and droping them
correlated_col = corr_columns(tmp_data,0.7)
correlated_col

In [ ]:
data_final.drop_duplicates(keep='first',inplace=True)

In [ ]:
YN_col = ['FULL_TIME_POSITION','SECONDARY_ENTITY_1','AGENT_REPRESENTING_EMPLOYER','H-1B_DEPENDENT','WILLFUL_VIOLATOR']
for col in YN_col:
    data_final[col] = data_final[col].map({'Y':1,'N':0},na_action='ignore')
data_final.NEW_EMPLOYMENT = pd.to_numeric(data_final.NEW_EMPLOYMENT,errors='coerce')
data_final.CONTINUED_EMPLOYMENT = pd.to_numeric(data_final.CONTINUED_EMPLOYMENT,errors='coerce')

In [ ]:
data_final.reset_index(drop=True, inplace = True)
data_final.to_csv('Final_updated_data.csv',index=False)